1RSB
===

\begin{align*}
p\left(f\right)  &= \sum_{f_{1},\dots,f_{k}}\delta\left(f;\min_{j=1,\dots,k}\left|f_{j}\right|\prod_{j=1}^{k}\text{sign}\left(f_{j}\right)\right)\prod_{j=1}^{k}q_j\left(f_{j}\right)e^{-y F_{ai}}\\
q(f) &= \sum_{f_1,\dots,f_d} \prod_{b=1,\dots,d} p_b(f_b) \delta\left(f-s-\sum_{b=1,\dots,d}f_b\right)e^{-y F_{ia}}\\
\end{align*}

where
\begin{align*}
F_{ai} &= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(-\prod_{j=1}^kf_{j}\right)\\
 F_{ia} &=\left|s + \sum_{b=1}^d f_b\right| - \sum_{b=1}^d|f_b|
\end{align*}



In [1]:
using OffsetArrays

const ∏ = prod
const ∑ = sum

sum (generic function with 14 methods)

In [2]:
using OffsetArrays

const ∏ = prod
const ∑ = sum


function iter_slow_factor(Q, J, y=0.0)
    k = length(Q)
    p = fill(0.0, -J:J)
    for fs in Iterators.product(fill(-J:J,k)...)
        Fai = -2*minimum(abs.(fs))*(∏(fs) < 0)
        w = ∏(q[f1] for (q,f1) ∈ zip(Q,fs)) * exp(-y*Fai)
        f = minimum(abs.(fs))*sign(∏(fs))
        p[clamp(f, -J, J)] += w
    end
    p ./ sum(p)
end

function iter_slow_var(P, s, J, y=0.0) 
    q = fill(0.0, -J:J)
    for fs in Iterators.product(fill(-J:J, length(P))...)
        f = sum(fs) + s
        Fia = abs(f) - sum(abs.(fs)) 
        w = ∏(p[f1] for (p,f1) ∈ zip(P,fs)) * exp(-y*Fia)
        q[clamp(f, -J, J)] += w 
    end
    q ./ sum(q)
end

iter_slow_var (generic function with 2 methods)

Simplifications
--

$F_{ai}$ can be simplified: 

\begin{align}
F_{ai} &= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(-\prod_{j=1}^kf_{j}\right)\\
&= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(\textrm{sign}\left(-\prod_{j=1}^kf_{j}\right)\right)\\
&= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(-\min_{j=1,\dots,k}|f_{j}|\textrm{sign}\left(\prod_{j=1}^kf_{j}\right)\right)\\
&= -2\left|\min_{j=1,\dots,k}|f_{j}|\textrm{sign}\left(-\prod_{j=1}^kf_{j}\right)\right|\Theta\left(-\min_{j=1,\dots,k}|f_{j}|\textrm{sign}\left(\prod_{j=1}^kf_{j}\right)\right)\\
&= 2f\Theta(-f)
\end{align}

So

\begin{align*}
p\left(f\right)= & \sum_{f_{1},\dots,f_{k}}\delta\left(f;\min_{j=1,\dots,k}\left|f_{j}\right|\prod_{j=1}^{k}\text{sign}\left(f_{j}\right)\right)\prod_{j=1}^{k}q_j\left(f_{j}\right)e^{-y 2 f \Theta(-f)}\\
\end{align*}

To compute $p$, define

\begin{align*}
a_k(f) &= p\left(\textrm{sign}\left(\prod_{i=1}^k f_i\right)=\textrm{sign}(f) \wedge |f_1|,\dots,|f_k| \ge |f|\right)\\
\end{align*}

$a_k$ satisfies the recursion

\begin{align*}
a_0(f) &= \delta(\textrm{sign}(f)-1)\\
a_k(f) &= a_{k-1}(f) \sum_{f_k\geq |f|}q_k(f_k) + a_{k-1}(-f) \sum_{f_k \leq -|f|} q_k(f_k)\\
\end{align*}

Then we finally have

$$p_k(f) = \cases{(a_k(f)-a_k(f+\textrm{sign}(f)))e^{-y2f\Theta(-f)} & for $f\neq0$\\
           1-\prod_{j=1}^k(1-q_j(0)) & for $f=0$}$$


In [3]:
function iter_factor(Q, J, y=0)
    a = fill(0.0, -J-1:J+1)
    a[1:J] .= 1
    for q ∈ Q
        Σp, Σm = 0.0, 0.0
        for f=J:-1:1
            ap, am = a[f], a[-f]
            Σp += q[f]; Σm += q[-f]
            #Σp, Σm = ∑(q[f:N]), ∑(q[-N:-f])
            a[+f] = ap*Σp + am*Σm
            a[-f] = am*Σp + ap*Σm
        end
    end
    
    p = fill(0.0, -J:J)
    for f = 1:J
        p[+f] = (a[+f]-a[+f+1])
        p[-f] = (a[-f]-a[-f-1])*exp(2y*f)
    end
    p[0] = 1-∏(1-q[0] for q ∈ Q)
    p ./ sum(p)
end


iter_factor (generic function with 2 methods)

Comparison
--

In [73]:
J=10
y=0.1
Q=[(p=fill(0.0,-J:J); p[-J:J] .= rand(2J+1); p ./=sum(p)) for i=1:1]
[iter_factor(Q,J,y) iter_slow_factor(Q,J,y)]

21×2 Matrix{Float64}:
 0.295408     0.295408
 0.134096     0.134096
 0.104974     0.104974
 0.030023     0.030023
 0.0146793    0.0146793
 0.00933844   0.00933844
 0.0897125    0.0897125
 0.0704443    0.0704443
 0.0185086    0.0185086
 0.0475206    0.0475206
 0.0147403    0.0147403
 0.023977     0.023977
 0.0198299    0.0198299
 0.0416744    0.0416744
 0.00275464   0.00275464
 0.00432038   0.00432038
 0.00765093   0.00765093
 0.0107772    0.0107772
 0.0132924    0.0132924
 0.000301125  0.000301125
 0.0459766    0.0459766

The expression for $F_{ia}$ is

\begin{align*}
    F_{ia} &=\left|s + \sum_{b=1}^d f_b\right| - \sum_{b=1}^d|f_b|
\end{align*}
and

\begin{align*}
q(f) &= \sum_{f_1,\dots,f_d} \prod_{b=1,\dots,d} p_b(f_b) \delta\left(f-s-\sum_{b=1}^df_b\right)e^{-y F_{ia}}\\
&= e^{-y |f|}\sum_{f_1,\dots,f_d} \prod_{b=1,\dots,d} p_b(f_b)e^{y|f_b|} \delta\left(f-s-\sum_{b=1}^df_b\right)\\
\end{align*}


To compute $q_d$, note that

\begin{align}
q(f) &= q_d(f)e^{-y|f|}
\end{align}

where $q_d$ satisfies
\begin{align}
q_0(f) & = \delta(f-s)\\
q_d(f) & = \sum_{f_d} q_{d-1}(f-f_d) p_d(f_d)e^{y|f_d|} 
\end{align}

In [5]:
function ⊛(p1, p2)
    q = fill(0.0,firstindex(p1)+firstindex(p2):lastindex(p1)+lastindex(p2))
    for f1 in eachindex(p1), f2 in eachindex(p2)
        q[f1+f2] += p1[f1]*p2[f2]
    end
    q
end

function iter_var(P, s, J, y=0)
    q = fill(1.0, s:s)
    for p ∈ P
        q = q ⊛ (p .* exp.(y .* abs.(eachindex(p))))
    end
    q .*= exp.(-y .* abs.(eachindex(q)))
    q2 = fill(0.0, -J:J)
    for f in eachindex(q)
        q2[clamp(f,-J,J)] += q[f]
    end
    q2 ./= sum(q2)
end

iter_var (generic function with 2 methods)

Comparison
--

In [7]:
J=13
y=1.0
s=1
P=[(p=fill(0.0, -J:J); p[-J:J] .= rand(2J+1); p ./=sum(p)) for i=1:3]
[iter_var(P,s,J,y) iter_slow_var(P,s,J,y)]

27×2 Matrix{Float64}:
 0.00250081  0.00250081
 0.014876    0.014876
 0.0175626   0.0175626
 0.031333    0.031333
 0.0419065   0.0419065
 0.0430645   0.0430645
 0.0526794   0.0526794
 0.0609549   0.0609549
 0.0738811   0.0738811
 0.0772615   0.0772615
 0.0911597   0.0911597
 0.100313    0.100313
 0.107176    0.107176
 ⋮           
 0.020032    0.020032
 0.017142    0.017142
 0.0156958   0.0156958
 0.0148698   0.0148698
 0.0131219   0.0131219
 0.0118257   0.0118257
 0.00922829  0.00922829
 0.00752562  0.00752562
 0.00564056  0.00564056
 0.00480792  0.00480792
 0.00460333  0.00460333
 0.00435273  0.00435273

In [18]:
using StatsBase, ProgressMeter

uni(J) = fill(1/(2J+1), -J:J)
residual(x) = (p=OffsetVector((x .* eachindex(x))[1:end], 0:lastindex(x)-1); p./=sum(p))

function RSB(Λ, K; 
        J=10, 
        maxiter=100, 
        popsize=1000, 
        popP = [uni(J) for i=1:popsize], 
        popQ = [uni(J) for i=1:popsize], 
        y=0)
    Λ1 = residual(Λ)
    K1 = residual(K)
    wΛ1 = weights(Λ1)
    wK1 = weights(K1)

    @showprogress for t = 1:maxiter
        for i = eachindex(popQ)
            d = sample(eachindex(Λ1), wΛ1)
            k = sample(eachindex(K1), wK1)
            Q = rand(popQ, k)
            P = rand(popP, d)
            s = rand((-1,1))
            popQ[i] = iter_var(P, s, J, y)
            popP[i] = iter_factor(Q, J, y)
        end
    end
    popP, popQ
end

RSB (generic function with 1 method)

In [57]:
Λ = OffsetVector([0,0,1], 0:2)
K = OffsetVector([0,0,0,1], 0:3)


J=10
popsize=10000
popQ = [OffsetVector(ones(2J+1)/(2J+1),-J:J) for i=1:popsize];
popP = [OffsetVector(ones(2J+1)/(2J+1),-J:J) for i=1:popsize];

In [59]:
popP, popQ = RSB(Λ,K; J=6, maxiter=1000, popsize=popsize, popQ=popQ, popP=popP, y=1.0);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:31


In [23]:
using StatsBase

proportions(argmax.(popP))

7-element Vector{Float64}:
 0.0009000000000000001
 0.0374
 0.24130000000000001
 0.4409
 0.2381
 0.0405
 0.0009000000000000001

1RSB equations for Max-Sum (Survey Propagation) at finite parameter $y$
---
We consider the auxiliary statistical physics model
\begin{align}
    \mathbb{P}_y(\{u_{ai},v_{ia}\})=\frac{1}{\Theta(y)}\mathbb{I}\left(\{u_{ai},v_{ia}\}\text{sat Max-Sum}\right)e^{-yF(\{u_{ai},v_{ia}\})}
\end{align}
in which the variables are the Max-Sum messages living on the edges, and the constraints (that can be put in a factorized form) enforces the Max-Sum equations to be satisfied. Each Max-Sum solution represents a cluster. The Max-Sum solutions are weighted according to their Bethe free energy, which corresponds to the minimal energy (inside the cluster). In the $y\to\infty$ limit one keeps only the clusters with minimal energy

* The distributions $Q_{ia}(v_{ia})$, $P_{ai}(u_{ai})$ over Max-Sum messages obey the SP(y) equations :
\begin{align}
    Q_{ia}(v_{ia})&=\frac{1}{Z_{ia}}\sum_{\{u_{bi}\}_{b\in\partial i\setminus a}}\delta(v_{ia}-f^{MS}_{ia}(\{u_{bi}\}_{b\in\partial i\setminus a}; s_i))e^{-yC_{ia}}\prod_{b\in\partial i\setminus a}P_{bi}(u_{bi}) \\
    P_{ai}(u_{ai})&=\frac{1}{Z_{ai}}\sum_{\{v_{ja}\}_{j\in\partial a\setminus i}}\delta(u_{ai}-f^{MS}_{ai}(\{v_{ja}\}_{j\in\partial a\setminus i}))e^{-yC_{ai}}\prod_{j\in\partial a\setminus i}Q_{ja}(v_{ja})    
\end{align}
where $v_{ia}-f^{MS}_{ia}(\{u_{bi}\}_{b\in\partial i\setminus a}; s_i)$, and $u_{ai}-f^{MS}_{ai}(\{v_{ja}\}_{j\in\partial a\setminus i})$ are shorthand notation for the Max-Sum equations
\begin{align}
    v_{ia}(\sigma_i)&=s_i\sigma_i+ \sum_{b\in\partial i\setminus a}u_{bi}(\sigma_i) - C_{ia}\\
    u_{ai}(\sigma_i)&=\max_{\{\sigma_j\}_{j\in\partial a\setminus i}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}v_{ja}(\sigma_j)\right) - C_{ai}
\end{align}
and $C_{ia}$, $C_{ai}$ are the constants in the Max-Sum equations
\begin{align}
    C_{ia}&=\max_{\sigma_i}\left(s_i\sigma_i+ \sum_{b\in\partial i\setminus a}u_{bi}(\sigma_i)\right)\\
    C_{ai}&=\max_{\{\sigma_i\}_{i\in\partial a}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}v_{ja}(\sigma_j)\right)
\end{align}
the constants $Z_{ia}$, $Z_{ai}$ are normalizations of the distributions $Q_{ia}$, $Q_{ai}$ over the Max-Sum messages

* With the parametrization $u_{ai}(\sigma_i) = f_{ai}\sigma_i+f_{0,ai}$ and $v_{ia}(\sigma_i) = g_{ia}\sigma_i + g_{0,ia}$ we get :
\begin{equation}
    f_{0,ai} = -\frac{1}{\beta}\log(2{\rm cosh}(\beta f_{ai})\to-|f_{ai}|
\end{equation}
(and the same for $g_{0,ia}$). Therefore :
\begin{align*}
    C_{ia}&=\max_{\sigma_i}\left(\sigma_i\left(s_i+ \sum_{b\in\partial i\setminus a}f_{bi}\right)\right) + \sum_{b\in\partial i\setminus a}f_{0,bi}\\
    &=\left|s_i+ \sum_{b\in\partial i\setminus a}f_{bi}\right| - \sum_{b\in\partial i\setminus a}|f_{bi}|
\end{align*}
and
\begin{align*}
    C_{ai}&=\max_{\{\sigma_i\}_{i\in\partial a}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}g_{ja}\sigma_j\right)-\sum_{j\in\partial a\setminus i}|g_{ja}|\\
    &= -2\min_{j\in\partial a\setminus i}|g_{ja}|\Theta\left(-\prod_{j\in\partial a\setminus j}g_{ja}\right)
\end{align*}

* We can then write the same 1RSB equations for the distributions $Q_{ia}(g_{ia})$, $P_{ai}(f_{ai})$ over the integer parameters $g_{ia}$, $f_{ai}$:
\begin{align}
    P(f)&=\frac{1}{Z_P(y)}\sum_{g_1,\dots g_k}\delta\left(f-\min_{j=1\dots,k}|g_j|\prod_j{\rm sgn}(g_j)\right)e^{-yC_{ai}(g_1,\dots,g_k)}\prod_j Q_j(g_j) \\
    Q(g)&=\frac{1}{Z_Q(y)}\sum_{f_1,\dots f_d}\delta(g-s-\sum_{b=1}^df_b)e^{-yC_{ia}(f_1,\dots,f_d)}\prod_{b=1}^dP_b(f_b)
\end{align}

* Random graph ensemble
When the source and the factor graph are random variables the 1RSB messages $P_{ai}$, $(a,i)\in E$ become random variables. Let $(a,i)$ be a uniformly chosen edge in a factor graph drawn from the random ensembl, and let $\mathcal{P}$ be the distribution of the message $P_{ai}$ solution of the 1RSB equation written above. The distribution $\mathcal{P}(p)$ obey consistency equation similar to the RS cavity equations:
\begin{align}
    \mathcal{P}^{1RSB}(P)&=\sum_{k}\tilde{P}_k \int\prod_{j=1}^k{\rm d}\mathcal{Q}^{1RSB}(Q_j)\delta(P-F^{SP(y)}(Q_1,\dots,Q_k)) \\
    \mathcal{Q}^{1RSB}(Q) &= \sum_{s}\frac{1}{2}\sum_{d}\tilde{\Lambda_d}\int\prod_{b=1}^d{\rm d}\mathcal{P}^{1RSB}(P_b)\delta(Q-G^{SP(y)}(P_1,\dots,P_d;s))
\end{align}
This equation always admits a trivial fixed point $\mathcal{P}(P)=\sum_f p^{RS}(f)\delta[P-\delta(\cdot-f)]$, $\mathcal{Q}(g)=\sum_g q^{RS}(g)\delta[Q-\delta(\cdot-g)]$, where $p^{RS}, q^{RS}$  is the solution of the RS cavity equation.
In the RS phase, this trivial fixed-point is the unique solution, while in the 1RSB phase, the trivial solution becomes unstable and the above equation admits a non-trivial solution.

Average minimal energy : Free energy of the auxiliary model
---
We want to estimate the Free energy $\mathcal{F}(y)=-\frac{1}{yn}\log\Theta(y)$ of the auxiliary problem defined by the partition function $\Theta(y)$ on a single instance. Then we can average over the random ensemble of instances. In the large $y\to\infty$ limit, we obtain the averaged minimal energy.

On a single instance, once a fixed point of the SP(y) equations has been found on a single instance, one can compute the Bethe free energy of the auxiliary problem:
\begin{align}
    \mathcal{F}(y) = \frac{1}{n}\sum_{a}\mathcal{F}_a(\{Q_{ia}\}_{i\in\partial a};y) + \frac{1}{n}\sum_i\mathcal{F}_i(\{P_{ai}\}_{a\in\partial i};y) - \frac{1}{n}\sum_{(ia)}\mathcal{F}_{(ia)}(P_{ai},Q_{ia};y)
\end{align}
with:
\begin{align}
    \mathcal{F}_a(\{Q_{ia}\}_{i\in\partial a};y) &= -\frac{1}{y}\log\left(\sum_{\{g_{ia}\}_{i\in\partial a}}e^{-yF_a(\{g_{ia}\}_{i\in\partial a})}\prod_{i\in\partial a}Q_{ia}(g_{ia})\right)\\
    \mathcal{F}_i(\{P_{ai}\}_{a\in\partial i};y) &= -\frac{1}{y}\log\left(\sum_{\{f_{ai}\}_{a\in\partial i}}e^{-yF_i(\{f_{ai}\}_{a\in\partial i})}\prod_{a\in\partial i}P_{ai}(f_{ai})\right)\\
    \mathcal{F}_{(ia)}(P_{ai},Q_{ia};y) &= -\frac{1}{y}\log\left(\sum_{f_{ia}, g_{ia}}e^{-yF_{ia}(f_{ia}, g_{ia})}P_{ai}(f_{ai})Q_{ia}(g_{ia})\right)
\end{align}
where $F_a,F_i, F_{ia}$ are the factor, variable and edge terms of the Bethe free energy associated to one fixed point of the Max-Sum equation: 
\begin{align}
    F_a(\{g_{ia}\}_{i\in\partial a}) &= 2\min_{i\in\partial a}|g_{ia}|\Theta(-\prod_i g_{ia})\\
    F_i(\{f_{ai}\}_{a\in\partial i}) &= -|s_i + \sum_{a\in\partial i}f_{ai}| + \sum_a |f_{ai}|\\
    F_{(ia)}(f_{ai},g_{ia}) &= -|f_{ai} + g_{ia}| + |f_{ai}| + |g_{ia}| 
\end{align}
Averaging over the random ensemble of instances:
\begin{align}
    \mathcal{F}^{1RSB}(y) &= \alpha\sum_{k}P_k\int\prod_{i=1}^k{\rm d}\mathcal{Q}^{1RSB}(Q_i)\mathcal{F}_a(Q_1,\dots,Q_k;y) + \sum_d\Lambda_d\sum_s\frac{1}{2}\int\prod_{i=1}^d\mathcal{P}^{1RSB}(P_i)\mathcal{F}_i(P_1,\dots,P_d;s,y) \\
    &- \Lambda'(1)\int\mathcal{F}_{(ia)}\mathcal{P}^{1RSB}(P)\mathcal{Q}^{1RSB}(Q)(P,Q;y)
\end{align}


In [44]:
function overlap_slow_factor(Q, J, y=0.0)
    w = 0.0
    k = length(Q)
    for fs in Iterators.product(fill(-J:J,k)...)
        Fa = 2*minimum(abs.(fs))*(∏(fs) < 0)
        w += ∏(q[f1] for (q,f1) ∈ zip(Q,fs)) * exp(-y*Fa)
    end
    log(w)
end

function overlap_slow_var(P, s, J, y=0.0)
    w = 0.0
    d = length(P)
    for fs in Iterators.product(fill(-J:J,d)...)
        f = sum(fs) + s
        Fi = -abs(f) + sum(abs.(fs)) 
        w += ∏(p[f1] for (p,f1) ∈ zip(P,fs)) * exp(-y*Fi)
    end
    log(w)
end

function overlap_slow_edge(p, q, J, y=0.0)
    w = 0.0
    for f in -J:J
        for g in -J:J
            Fia = abs(f)+abs(g)-abs(f+g)
            w += p[f]*q[g]*exp(-y*Fia)
        end
    end
    log(w)
end

overlap_slow_edge (generic function with 2 methods)

In [74]:
function overlap1RSB(Λ, K; 
        popP, 
        popQ, 
        samples=length(popP), 
        y=0.0)

    J = lastindex(popQ[1])
    wΛ = weights(Λ)
    wK = weights(K)

    O_factor = 0.0
    O_var = 0.0
    O_edge = 0.0
    
    @showprogress for t = 1:samples
        k = sample(eachindex(K), wK)
        Q = rand(popQ, k)
        O_factor += overlap_slow_factor(Q, J, y)/samples

        d = sample(eachindex(Λ), wΛ)
        P = rand(popP, d)
        s = rand((-1,1))
        O_var += overlap_slow_var(P, s, J, y)/samples

        p = rand(popP)
        q = rand(popQ)
        O_edge += overlap_slow_edge(p, q, J, y)/samples
    end
    mK = sum(k*K[k] for k=eachindex(K))
    mΛ = sum(d*Λ[d] for d=eachindex(Λ))
    α = mΛ/mK
    -1/y*(-α*O_factor - O_var + mΛ*O_edge)
end

overlap1RSB (generic function with 1 method)

In [75]:
Λ = OffsetVector([0,0,1], 0:2)
K = OffsetVector([0,0,0,1], 0:3)
popP, popQ = RSB(Λ,K; J=6, maxiter=10000, popsize=popsize, y=1.0);

Progress: 100%|█████████████████████████████████████████| Time: 0:04:41


In [76]:
O = overlap1RSB(Λ,K; popP=popP, popQ=popQ, y=1.0, samples=10^5)

Progress: 100%|█████████████████████████████████████████| Time: 0:06:07m


0.581780000000194

In [77]:
D=(1-O)/2

0.20910999999990298